# Imports

In [1]:
import numpy as np
import pandas as pd
import requests
from io import BytesIO, StringIO
import os

## Download and prepare a labelled dataset with the url of the images

In [4]:
url = "http://world.openfoodfacts.org/data/en.openfoodfacts.org.products.csv"
s = requests.get(url).content
whole_data = pd.read_csv(StringIO(s.decode('utf-8')), sep='\t')

#Keep two brands only: Carrefour = 1 and Auchan = 0
two_classes_data = whole_data.query("brands == 'Carrefour'|brands == 'Auchan'")
two_classes_data = two_classes_data.reset_index()
two_classes_data = two_classes_data.loc[:,['brands','image_url',
                                           #'image_small_url'
                                          ]
                                       ]

#two_classes_data.brands.value_counts()
two_classes_data.brands = two_classes_data.brands.str.replace('Carrefour', '1')
two_classes_data.brands = two_classes_data.brands.str.replace('Auchan', '0')

two_classes_data = two_classes_data.loc[two_classes_data.image_url.notnull(),:]
two_classes_data = two_classes_data.reset_index()
del two_classes_data['index']

/home/kevin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
npixel = (300,400)
wd_path = '/home/kevin/Desktop/OpenFoodFacts/OpenFoodFacts/data/csv'

## Define an auxiliary function to download the images from their url

In [2]:
path = '/home/kevin/Desktop/OpenFoodFacts/OpenFoodFacts/data/images'

def download_url(image_url, image_name, directory_path = path):
    """
        Downloads the images from url into selected directory
        
        Arguments:
            *url: the url of the site
            *image_name: the name to use to save the image
            *directory_path: the path of the directory where to save
        
        Returns:
        
    """
    r = requests.get(image_url)
    pic = Image.open(BytesIO(r.content))
    pic.save(os.path.join(path, image_name))
    print("Saving the image {} in working directory \n".format(image_name))


# Download with the right name and save in numpy format

In [ ]:
imgs = np.empty(shape = (0, npixel[1], npixel[0], 3), dtype = np.uint8)
labels = np.empty(shape = (0,), dtype = np.bool)

data = two_classes_data.head(10)
for row in data.itertuples():
    if row.brands == '1':
        image_name = 'carrefour{}.jpg'.format(str(row.Index))
    else:
        image_name = 'auchan{}.jpg'.format(str(row.Index))
    image_url = row.image_url
    x = image_url_to_nparray(image_url,image_name)
    print(row)
    imgs = np.vstack([imgs, x])
    labels = np.concatenate((labels, [row.brands]))

np_path = '/home/kevin/Desktop/OpenFoodFacts/OpenFoodFacts/data/npy'
imgs_np_path = os.path.join(np_path, 'images.npy')
labels_np_path = os.path.join(np_path, 'labels.npy')
np.save(imgs_np_path, imgs)
np.save(labels_np_path, labels)

## Load them into a numpy array

In [ ]:
np_path = '/home/kevin/Desktop/OpenFoodFacts/OpenFoodFacts/data/npy'
imgs_np_path = os.path.join(np_path, 'images.npy')
labels_np_path = os.path.join(np_path, 'labels.npy')
Y = np.load(labels_np_path)
X = np.load(imgs_np_path)